# Phân đoạn Pneumothorax (tràn khí màng phổi)

## Tổng quan bài toán

Trong dự án này, sẽ phải phân đoạn được đâu là phần tràn khí màng phổi từ hình ảnh X-quang đã được gán nhãn trước.

Dữ liệu bao gồm có 12955 hình X-quang từ 12047 bệnh nhân và nhãn tương ứng (có những hình ảnh có nhãn, có những hình ảnh không có nhãn tức là không bị chứng tràn khí màng phổi).

## Mục tiêu

* Đào tạo mô hình segment dựa trên Unet với các backbone:
    * Resnet50
    * Efficientnet
    * Xceptionnet
* Thử nghiệm các phương pháp augumentation trên hình ảnh X-Quang
* Đánh giá mô hình:
    * Trên metrics dice score
    * Đánh giá cho từng mô hình với các backbone khác nhau

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as image
from tqdm.notebook import tqdm
import glob
import pydicom
import sys
import os

print(os.listdir("../input/siim-acr-pneumothorax-segmentation"))
print()
sys.path.insert(0, '../input/siim-acr-pneumothorax-segmentation')

from mask_functions import rle2mask
%matplotlib inline

In [ ]:
# install libraries for augumentation image and
# visualize architecture model Unet
!pip install albumentations
!pip install torchsummary

In [ ]:
import torch
import torchvision
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset

from torchsummary import summary
import torchvision.models as models
import torchvision.transforms as T
import albumentations as A

from torch.autograd import Variable

## Load information for dataset

In [ ]:
# Load information for dataset
train_df = pd.read_csv("../input/segment-data/train_info_split.csv")
val_df = pd.read_csv("../input/segment-data/val_info_split.csv")

In [ ]:
# Load all path for file .dcm
file_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/*/*/*.dcm"
file_paths = glob.glob(file_path)

## Visualize image origin and mask with file .dcm

In [ ]:
# Visualize image mask for file .dcm
image_id_arr = train_df["ImageId"].unique()

for index, image_id in enumerate(image_id_arr):
    index_ = list(filter(lambda x: image_id in file_paths[x], range(len(file_paths))))
    dataset = pydicom.dcmread(file_paths[index_[0]])
    image_data = dataset.pixel_array
    
    record_arr = train_df[train_df["ImageId"]==image_id]
    # Visualize patient has multi segment
    if len(record_arr) >= 2:
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_figheight(15)
        fig.set_figwidth(15)
        ax1.imshow(image_data, cmap=plt.cm.bone)
        ax2.imshow(image_data, cmap=plt.cm.bone)
        mask = np.zeros((1024, 1024))
        for _, row in record_arr.iterrows():
            if row["EncodedPixels"] != ' -1':
                mask_ = rle2mask(row["EncodedPixels"], 1024, 1024).T
                mask[mask_==255] = 255
        
        ax2.imshow(mask, alpha=0.3, cmap="Blues")    
        break

## Khởi tạo kiến trúc mô hình

### Các hàm trợ giúp

In [ ]:
def toTensor(np_array, axis=(2,0,1)):
    return torch.tensor(np_array).permute(axis)

def toNumpy(tensor, axis=(1,2,0)):
    return tensor.detach().cpu().permute(axis).numpy()

### Tạo Data Loader

In [ ]:
# Visualize image mask for file .dcm
image_id_arr = train_df["ImageId"].unique()

for index, image_id in enumerate(image_id_arr):
    index_ = list(filter(lambda x: image_id in file_paths[x], range(len(file_paths))))
    dataset = pydicom.dcmread(file_paths[index_[0]])
    image_data = dataset.pixel_array
    
    record_arr = train_df[train_df["ImageId"]==image_id]
    # Visualize patient has multi segment
    if len(record_arr) >= 2:
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_figheight(15)
        fig.set_figwidth(15)
        ax1.imshow(image_data, cmap=plt.cm.bone)
        ax2.imshow(image_data, cmap=plt.cm.bone)
        mask = np.zeros((1024, 1024))
        for _, row in record_arr.iterrows():
            if row["EncodedPixels"] != ' -1':
                mask_ = rle2mask(row["EncodedPixels"], 1024, 1024).T
                mask[mask_==255] = 255
        
        ax2.imshow(mask, alpha=0.3, cmap="Blues")    
        break

In [ ]:
def get_infor(df):
    infor = []
    image_id_arr = df["ImageId"].unique()
    for index, image_id in tqdm(enumerate(image_id_arr)):
        index_ = list(filter(lambda x: image_id in file_paths[x], range(len(file_paths))))
        full_image_path = file_paths[index_[0]]

        # Get all segment encode
        record_arr = train_df[train_df["ImageId"]==image_id]
        encode_pixels = []
        for _, row in record_arr.iterrows():
            encode_pixels.append(row["EncodedPixels"])

        infor.append({
            "key": image_id,
            "file_path": full_image_path,
            "mask": encode_pixels
        })
    return infor

print("Loading information for training set \n")
train_infor = get_infor(train_df)
print("Loading information for validation set \n")
val_infor = get_infor(val_df)

In [ ]:
import cv2

class MaskDataset(Dataset):
    def __init__(self, df, img_info, transforms=None):
        self.df = df
        self.img_info = img_info
        self.transforms = transforms
        
    def __getitem__(self, idx):
        img_path = self.img_info[idx]["file_path"]
        key = self.img_info[idx]["key"]
        
        # load image data
        dataset = pydicom.dcmread(img_path)
        img = dataset.pixel_array
        img = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
        
        mask_arr = self.img_info[idx]["mask"]
        
        mask = np.zeros((512, 512))
        
        for item in mask_arr:
            if item != " -1":
                mask_ = rle2mask(item, 1024, 1024).T
                mask_ = cv2.resize(mask_, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
                mask[mask_==255] = 255

        # to Tensor
        mask = np.expand_dims(mask, axis=-1)/255.0
        mask = toTensor(mask).float()
        
        img = np.expand_dims(img, axis=-1)/255.0
        img = toTensor(img).float()
        
        return img, mask
            
    def __len__(self):
        return len(self.img_info)

In [ ]:
train_dataset = MaskDataset(train_df, train_infor)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, drop_last=True)

val_dataset = MaskDataset(val_df, val_infor)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True, drop_last=True)

number_visualize = 1
for img, mask in train_dataset:
    if number_visualize > 5:
        break
    img = toNumpy(img)[:,:,0]
    mask = toNumpy(mask)[:,:,0]

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    ax1.imshow(img, cmap=plt.cm.bone)
    ax2.imshow(img, cmap=plt.cm.bone)
    ax2.imshow(mask, alpha=0.3, cmap="Blues")
    number_visualize += 1


### Định nghĩa hàm đo dice score và tính toán dice loss

Đọc trong: [jeremyjordan semantic-segmentation](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
!pip install torchsummary
import os
import cv2
import pandas
import numpy as np
from tqdm import tqdm
from glob import glob

import matplotlib
import matplotlib.pyplot as plt
from random import choice, choices, shuffle

import torch
import torchvision
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset

from torchsummary import summary
import torchvision.models as models
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from random import randint
import albumentations as A
from PIL import Image


In [ ]:
def Deconv(n_input, n_output, k_size=4, stride=2, padding=1):
    Tconv = nn.ConvTranspose2d(
        n_input, n_output,
        kernel_size=k_size,
        stride=stride, padding=padding,
        bias=False)
    block = [
        Tconv,
        nn.BatchNorm2d(n_output),
        nn.LeakyReLU(inplace=True),
    ]
    return nn.Sequential(*block)
        

def Conv(n_input, n_output, k_size=4, stride=2, padding=0, bn=False, dropout=0):
    conv = nn.Conv2d(
        n_input, n_output,
        kernel_size=k_size,
        stride=stride,
        padding=padding, bias=False)
    block = [
        conv,
        nn.BatchNorm2d(n_output),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(dropout)
    ]
    return nn.Sequential(*block)

class Unet(nn.Module):
    def __init__(self, resnet):
        super().__init__()
        
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
        # get some layer from resnet to make skip connection
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        # convolution layer, use to reduce the number of channel => reduce weight number
        self.conv_5 = Conv(2048, 512, 1, 1, 0)
        self.conv_4 = Conv(2048, 1024, 1, 1, 0)
        self.conv_3 = Conv(1024, 512, 1, 1, 0)
        self.conv_2 = Conv(512, 128, 1, 1, 0)
        self.conv_1 = Conv(128, 64, 1, 1, 0)
        self.conv_0 = Conv(32, 1, 1, 1, 0)
        
        # deconvolution layer
        self.deconv4 = Deconv(512, 1024, 3, 2, 1)
        self.deconv3 = Deconv(1024, 512, 3, 2, 1)
        self.deconv2 = Deconv(512, 256, 3, 2, 1)
        self.deconv1 = Deconv(128, 64, 4, 2, 1)
        self.deconv0 = Deconv(64, 32, 2, 2, 2)
        
        
    def forward(self, x):
        # down sample
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        skip_1 = x
        
        x = self.maxpool(x)
        x = self.layer1(x)
        skip_2 = x

        x = self.layer2(x)
        skip_3 = x
        x = self.layer3(x)
        skip_4 = x
        
        x5 = self.layer4(x)
        x5 = self.conv_5(x5)
        
        # up sample
        x4 = self.deconv4(x5)
        x4 = torch.cat([x4, skip_4], dim=1)
        x4 = self.conv_4(x4)
        
        x3 = self.deconv3(x4)
        x3 = torch.cat([x3, skip_3], dim=1)
        x3 = self.conv_3(x3)
        x2 = self.deconv2(x3)
        
        x2 = torch.cat([x2, skip_2], dim=1)
        x2 = self.conv_2(x2)
        
        x1 = self.deconv1(x2)
        x1 = torch.cat([x1, skip_1], dim=1)
        x1 = self.conv_1(x1)
        
        x0 = self.deconv0(x1)
        x0 = self.conv_0(x0)
        x0 = self.sigmoid(x0)
        return x0

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_ft = models.resnet50(pretrained=True)
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)

model = Unet(model_ft)
model.to(device)

for i, child in enumerate(model.children()):
    if i <= 7:
        for param in child.parameters():
            param.requires_grad = False

print(summary(model,input_size=(1,512,512)))

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

def get_dice_score(inputs, targets, smooth=1):
    intersection = (inputs * targets).sum((3,2,1))                            
    dice_scores = (2.*intersection + smooth)/(inputs.sum((3,2,1)) + targets.sum((3,2,1)) + smooth)   
    dice_score = dice_scores.mean()
    
    return dice_score, dice_scores

def loss_function(inputs, targets, smooth=1):
    # soft dice loss
    intersection = (inputs * targets).sum((3,2,1))                            
    dice = (2.*intersection + smooth)/((inputs**2).sum((3,2,1)) + (targets**2).sum((3,2,1)) + smooth)   
    dice_loss = 1 - dice.mean()
    # binary cross entropy
    BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
    # overall loss
    loss = dice_loss + BCE
    return loss

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

# free_gpu_cache()

In [ ]:
import csv

train_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.Adam(train_params, lr=0.001, betas=(0.9, 0.99))

epochs = 100
model.train()
saved_dir = "model"
os.makedirs(saved_dir, exist_ok=True)
path_csv_history = "./model/history.csv"

with open(path_csv_history, mode='w') as csv_file:
    fieldnames = ['train_dice_score', 'val_dice_score']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for epoch in range(epochs):
        number_iter = 0
        train_dice_score = []
        for imgs, masks in tqdm(train_loader):
            optimizer.zero_grad()
            imgs_gpu = imgs.to(device)
            outputs = model(imgs_gpu)
            # outputs = F.sigmoid(outputs) 
            masks = masks.to(device)
            
            _, dice_scores = get_dice_score(outputs, masks)
            
            loss = loss_function(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_dice_score.extend(dice_scores)
        train_dice_score = torch.mean(torch.stack(train_dice_score))
        print(f"Epoch {epoch}, Dice score in training set: {train_dice_score:0.4f}")
            
        # free_gpu_cache()
        val_dice_score = []
        with torch.no_grad():
            for imgs, masks in tqdm(val_loader):
                imgs_gpu = imgs.to(device)
                outputs = model(imgs_gpu)
                outputs = F.sigmoid(outputs) 
                masks = masks.to(device)

                _, dice_scores = get_dice_score(outputs, masks)
                val_dice_score.extend(dice_scores)
            val_dice_score = torch.mean(torch.stack(val_dice_score))
            print(f"Epoch {epoch}, Dice score in validation set: {val_dice_score:0.4f}")
            path = "./model/model_unet_epoch_{:.1f}_dice_score_{:0.4f}".format(epoch, val_dice_score.mean())
            torch.save(model.state_dict(), path)
        
        writer.writerow({
            "train_dice_score": train_dice_score,
            "val_dice_score": val_dice_score
        })